<a href="https://colab.research.google.com/github/edgarbc/LLM_optimizer/blob/main/my_DSPy_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

My DSPy tutorial

Example of how to use DSPy for systematic optimization of a simple LLM application.

Adapted from [this blog post](https://learnbybuilding.ai/tutorials/a-gentle-introduction-to-dspy).

Edgar Bermudez

May 2024.

In [1]:
# load the credentials and secrets variables
from dotenv import load_dotenv
load_dotenv()

ModuleNotFoundError: No module named 'dotenv'

In [ ]:
!pip install dspy

In [ ]:
!pip install kaggle

## Web page information

For this example we are going to extract about the current Kaggle competitions. Kaggle is a learning-by-doing platform in which challenges are posted as a competition. Users submit ML solutions that are evaluated and ranked. At the end of the competitions the top-leaderboard winners get a money prize (some money-free prizes are available). In general, it is a great place to learn from the community.

Now you need to get the kaggle api key from your kaggle profile to be able to get kaggle data.

In [5]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 63 bytes


In [6]:
import kaggle

In [7]:
!kaggle competitions list

ref                                                                                deadline             category             reward  teamCount  userHasEntered  
---------------------------------------------------------------------------------  -------------------  ---------------  ----------  ---------  --------------  
https://www.kaggle.com/competitions/ai-mathematical-olympiad-prize                 2024-06-27 23:59:00  Featured         $1,048,576        550           False  
https://www.kaggle.com/competitions/home-credit-credit-risk-model-stability        2024-05-27 23:59:00  Featured           $105,000       3364           False  
https://www.kaggle.com/competitions/lmsys-chatbot-arena                            2024-08-05 23:59:00  Research           $100,000        228           False  
https://www.kaggle.com/competitions/learning-agency-lab-automated-essay-scoring-2  2024-07-02 23:59:00  Featured            $50,000       1317           False  
https://www.kaggle.com/competition

In [ ]:
import requests
from bs4 import BeautifulSoup
import dspy

website_url = "https://www.kaggle.com/competitions"

#website_url = "https://www.kaggle.com/competitions/playground-series-s4e5/"

res = requests.get(website_url)
soup = BeautifulSoup(res.text, 'html.parser')
raw_text = [p.text for p in soup.find_all('p') if p.text]

In [ ]:
# now show some of the extracted text
raw_text[:10]

print(soup)